In [1]:
import os
import cv2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
from sklearn.cluster import KMeans
from utils import *
from feature_util import *
from sklearn import preprocessing

In [2]:
#load data
all_images, all_labels = load_images_and_augment()
X_train, X_val, X_test, y_train, y_val, y_test = train_test_validation_split(all_images,all_labels)

In [44]:
#encodes the labels
label_encoder = preprocessing.LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_val_encoded = label_encoder.fit_transform(y_val)
y_test_encoded = label_encoder.fit_transform(y_test)
n_clusters = np.unique(y_train_encoded)

In [45]:
#flattening dataset
flat_X_train = []
for image in (X_train):
    flat_image = np.ravel(image)
    flat_X_train.append(flat_image)

In [46]:
#Performing KMeans on the grayscaled images
kmeans = KMeans(n_clusters = 4, random_state = 10, init = 'k-means++', n_init = 5).fit(flat_X_train)

In [47]:
#getting the labels of each of the images
testing = kmeans.labels_

#ground truth labels
testing_gt = y_train_encoded
print(testing)
print(testing_gt)

[3 2 1 ... 0 0 2]
[2 0 3 ... 3 1 0]


In [48]:
#accuracy metric
acc_count = 0
for i in range(len(y_test_encoded)):
    if testing[i] == testing_gt[i]:
        acc_count += 1

total_accuracy = acc_count/len(y_test_encoded)
print(total_accuracy)
    

0.24470734744707348


In [49]:
#KMEANS on KEY FEATURE POINTS

#getting the main key points for each image
image_kp = []
image_labels = []
kp_len = []

for i in range(len(X_train)):

    image  = X_train[i]
    image = cv2.normalize(image,  None, 0, 255, cv2.NORM_MINMAX)
    image = image.astype(np.uint8)
    label = y_train[i]
    pixels = key_point_pixels(image, 30)[0]
    v_lengths = key_point_pixels(image, 30)[1]
    kp_len.append(v_lengths)
    image_kp.append(pixels)
    image_labels.append(label)

len(image_kp)

12846

In [50]:
#finding the images with exactly 30 keypoints
min_30_kp = []
label_30_kp = []

for i in range(len(image_kp)):
    if kp_len[i] == 30:
        min_30_kp.append(image_kp[i])
        label_30_kp.append(image_labels[i])

In [51]:
#Performing KMeans on the pixels that were considered to be key feature points
kmeans_kp =  KMeans(n_clusters = 4, random_state = 10, init = 'k-means++', n_init = 5).fit(min_30_kp)

In [54]:
#encoding the labels to do th comparison
label_30_kp_encoded = label_encoder.fit_transform(label_30_kp)

#getting the labels of each of the images
kp_test = kmeans_kp.labels_

#ground truth labels
kp_test_gt = label_30_kp_encoded


print(kp_test[:10])
print(kp_test_gt[:10])

[3 3 2 1 0 2 2 0 3 3]
[2 0 3 2 2 0 1 0 3 3]


In [56]:
#calculating the accuracy of the clustering
kp_accuracy_count = 0
for i in range(len(kp_test)):
    if kp_test[i] == kp_test_gt[i]:
        kp_accuracy_count += 1

total_kp_accuracy = kp_accuracy_count/len(kp_test_gt)
print(total_kp_accuracy)

0.25465628174737553
